In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/main-training-data-csv/main_training_data.csv


In [ ]:
df = pd.read_csv("/kaggle/input/main-training-data-csv/main_training_data.csv")


In [3]:
# Separate target
y = df["target"].astype(np.int8)

# Keep user_id for splitting ONLY
user_ids = df["user_id"]

# Drop non-feature columns
X = df.drop(columns=["target", "user_id", "product_id"])

# Downcast to float32
X = X.astype(np.float32)


In [4]:
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 17 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   user_total_orders             float32
 1   user_total_products           float32
 2   user_avg_basket_size          float32
 3   user_reorder_ratio            float32
 4   user_avg_days_between_orders  float32
 5   prod_total_purchases          float32
 6   prod_unique_users             float32
 7   prod_reorder_rate             float32
 8   prod_avg_cart_position        float32
 9   prod_order_count              float32
 10  up_times_bought               float32
 11  up_first_order_number         float32
 12  up_last_order_number          float32
 13  up_order_rate                 float32
 14  up_reorder_ratio              float32
 15  up_avg_cart_position          float32
 16  up_orders_since_last_bought   float32
dtypes: float32(17)
memory usage: 863.0 MB


In [6]:
from sklearn.model_selection import train_test_split

unique_users = user_ids.unique()

train_users, val_users = train_test_split(
    unique_users,
    test_size=0.2,
    random_state=42
)


In [7]:
train_mask = user_ids.isin(train_users)
val_mask   = user_ids.isin(val_users)

X_train, X_val = X[train_mask], X[val_mask]
y_train, y_val = y[train_mask], y[val_mask]


In [8]:
set(user_ids[train_mask]).intersection(set(user_ids[val_mask]))


set()

In [9]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,
    class_weight="balanced"
)

logreg.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000, n_jobs=-1)

In [10]:
from sklearn.metrics import roc_auc_score, log_loss
val_preds = logreg.predict_proba(X_val)[:, 1]

auc = roc_auc_score(y_val, val_preds)
ll  = log_loss(y_val, val_preds)

print("Validation AUC:", auc)
print("Validation LogLoss:", ll)


Validation AUC: 0.79356256398605
Validation LogLoss: 0.5649207414772535


In [12]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# unique users
unique_users = df["user_id"].unique()

train_users, val_users = train_test_split(
    unique_users,
    test_size=0.1,
    random_state=42
)

train_mask = df["user_id"].isin(train_users)
val_mask   = df["user_id"].isin(val_users)

X_train_full = X[train_mask]
y_train_full = y[train_mask]

X_val_full = X[val_mask]
y_val_full = y[val_mask]


In [13]:
best_params = {
    "objective": "binary",
    "metric": "auc",
    "boosting_type": "gbdt",
    "verbosity": -1,

    "learning_rate": 0.010214245053789754,
    "num_leaves": 57,
    "max_depth": 7,
    "min_child_samples": 114,
    "feature_fraction": 0.9460679839301878,
    "bagging_fraction": 0.7950944412841229,
    "bagging_freq": 2,
    "lambda_l1": 4.853810799967207,
    "lambda_l2": 3.3396540347692834,

    "device": "gpu"
}


In [14]:
train_data = lgb.Dataset(X_train_full, label=y_train_full)
val_data   = lgb.Dataset(X_val_full, label=y_val_full)

final_lgb = lgb.train(
    best_params,
    train_data,
    valid_sets=[val_data],
    num_boost_round=2000,
    callbacks=[
        lgb.early_stopping(50),
        lgb.log_evaluation(100),
    ]
)


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 50 rounds
[100]	valid_0's auc: 0.811768
[200]	valid_0's auc: 0.813828
[300]	valid_0's auc: 0.814809
[400]	valid_0's auc: 0.81539
[500]	valid_0's auc: 0.815735
[600]	valid_0's auc: 0.81594
[700]	valid_0's auc: 0.816071
[800]	valid_0's auc: 0.816186
[900]	valid_0's auc: 0.81627
[1000]	valid_0's auc: 0.816355
[1100]	valid_0's auc: 0.816421
[1200]	valid_0's auc: 0.816476
[1300]	valid_0's auc: 0.816517
[1400]	valid_0's auc: 0.816573
[1500]	valid_0's auc: 0.816613
[1600]	valid_0's auc: 0.816634
[1700]	valid_0's auc: 0.816659
[1800]	valid_0's auc: 0.816679
[1900]	valid_0's auc: 0.816704
[2000]	valid_0's auc: 0.816717
Did not meet early stopping. Best iteration is:
[1992]	valid_0's auc: 0.816718
